In [1]:
from pyspark import SparkContext
from geopyspark import geopyspark_conf
from geopyspark.geotrellis.constants import LayerType, LayoutScheme
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis import Extent
from geopyspark_netcdf.datasets import Gddp
from shapely.geometry import shape

In [ ]:
sc = SparkContext(conf=geopyspark_conf(appName="Crenshaw Boulevard"))

In [ ]:
uri = "s3://nasanex/NEX-GDDP/BCSD/rcp85/day/atmos/tasmin/r1i1p1/v1.0/tasmin_day_BCSD_rcp85_r1i1p1_inmcm4_2099.nc"
# uri = "/tmp/tasmin_day_BCSD_rcp85_r1i1p1_inmcm4_2099.nc"

In [ ]:
!curl 'https://raw.githubusercontent.com/jamesmcclain/SlausonAvenue/master/geojson/CA.geo.json' -o /tmp/CA.geo.json

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage

%matplotlib inline

In [ ]:
geojson = json.loads(open("/tmp/CA.geo.json").read())
ca = shape(geojson['features'][0]['geometry'])
ca

In [ ]:
snippet = Gddp.raster(uri, ca.bounds, 7, sc)

In [ ]:
plt.imshow(snippet)

In [ ]:
M.set_center(-118.225, 33.897, 10)

In [ ]:
days = range(0, 365)

In [ ]:
raster_rdd = Gddp.rdd_of_rasters(uri, ca.bounds, days, 0, sc)

In [ ]:
masked_rdd = raster_rdd.mask(ca)

In [ ]:
cpt = list(Gddp.samples(uri, (-118.225, 33.897), range(0,365), sc).collect())
# um = list(Gddp.samples(uri, (-89.5363816, 34.3656821), range(0, 365), sc).collect())

In [ ]:
plt.plot(days, cpt, label="Hub City")
# plt.plot(days, um)
plt.legend(loc='best')

# Build GeoTiff from NumPy Array #

In [ ]:
import gdal

In [ ]:
snippet = scipy.ndimage.zoom(Gddp.raster(uri, ca.bounds, 33, sc), 128, order=3)
(rows, columns) = snippet.shape

In [ ]:
wkt = """GEOGCS[\"WGS 84\",
    DATUM[\"WGS_1984\",
        SPHEROID[\"WGS 84\",6378137,298.257223563,
            AUTHORITY[\"EPSG\",\"7030\"]],
        AUTHORITY[\"EPSG",\"6326\"]],
    PRIMEM[\"Greenwich\",0],
    UNIT[\"degree\",0.0174532925199433],
    AUTHORITY[\"EPSG",\"4326\"]]"""

## Build Transform ##

http://www.gdal.org/gdal_tutorial.html

```
adfGeoTransform[0] /* top left x */
adfGeoTransform[1] /* w-e pixel resolution */
adfGeoTransform[2] /* 0 */
adfGeoTransform[3] /* top left y */
adfGeoTransform[4] /* 0 */
adfGeoTransform[5] /* n-s pixel resolution (negative value) */
```

In [ ]:
(xmin, ymin, xmax, ymax) = ca.bounds
transform = (xmin, (xmax - xmin)/columns, 0, ymax, 0, (ymin - ymax)/rows)
transform

## Dump To GeoTiff ##

https://gis.stackexchange.com/questions/58517/python-gdal-save-array-as-raster-with-projection-from-other-file

In [ ]:
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create("/tmp/ca.tif", columns, rows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform(transform)
dataset.SetProjection(wkt)
dataset.GetRasterBand(1).WriteArray(snippet)
dataset.GetRasterBand(1).SetNoDataValue(-1.0)
dataset.FlushCache()

## Display ##

In [ ]:
from geonotebook.wrappers.raster import TMSRasterData

from geopyspark.geotrellis.geotiff import get
from geopyspark.geotrellis.tms import *
from geopyspark.geotrellis.color import ColorMap

Trouble with reproject, use `gdalwarp` to reproject to WebMercator.

In [ ]:
!rm -f /tmp/ca2.tif

In [ ]:
!~/local/gdal/bin/gdalwarp -t_srs 'EPSG:3857' /tmp/ca.tif /tmp/ca2.tif

In [ ]:
rdd = get(sc, LayerType.SPATIAL, "/tmp/ca2.tif", max_tile_size=512, num_partitions=32)

In [ ]:
metadata = rdd.collect_metadata(tile_size=512)

In [ ]:
laid_out = rdd.tile_to_layout(metadata)

In [ ]:
reprojected = laid_out.reproject("EPSG:3857", scheme=LayoutScheme.ZOOM)

In [ ]:
from functools import partial
import pyproj
from shapely.geometry import shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

ca2 = transform(project, ca)
ca2

In [ ]:
target = reprojected.mask(ca2)

In [ ]:
pyramid = target.pyramid(1, target.zoom_level)

In [ ]:
colormap = ColorMap.build(pysc=sc, breaks=target.get_histogram(), colors='viridis')

In [ ]:
data = rdd_tms_server(sc, pyramid, colormap)

In [ ]:
M.add_layer(TMSRasterData(data))

In [ ]:
M.remove_layer(M.layers[0])